In [1]:
import torch
import numpy as np
from models.training import easyTrainer, weights_to_dataset
from models.nODE import nODE, make_nODE_from_parameters
import matplotlib.pyplot as plt
from torch_geometric.data import Data
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.loader import DataLoader

In [2]:
adj_list = []
adj_list.append(np.array([[0.,0.],[0.,0.]]))
adj_list.append(np.array([[0,1.],[0,0]]))
adj_list.append(np.array([[0,-1.],[0,0]]))
adj_list.append(np.array([[0,1.],[1.,0]]))
adj_list.append(np.array([[0,1.],[-1.,0]]))
adj_list.append(np.array([[0,-1.],[-1.,0]]))

In [49]:
def generate_data(adj,y,ndata=4):
    ODE_dim = 2
    Gamma = np.array([-1., -1.])
    integration_time = 1
    Win = adj
    Wout = np.array([[2., 0], [0, 2.]])
    bin = np.array([[2.], [2.]])
    bout = np.array([[2.], [2.]])

    train_data, test_data = weights_to_dataset(integration_time, Gamma, Win=Win, bin=bin, Wout=Wout, bout=bout, batch_size = ndata)

    edge_index = torch.tensor([[0, 1],
                           [1, 0]], dtype=torch.long)
    
    for i, (x0, xt) in enumerate(train_data):
        x = torch.tensor([xt.t()[0].tolist(),xt.t()[1].tolist()], dtype=torch.float)
        y = torch.tensor(abs(adj).flatten())

        print(x)
        data = Data(x= x, edge_index=edge_index,y=y)

    return data

data_list = []
for graph_ind in [0,1,2,3,4,5]:
    for i in range(0,100):
        ind = 0
        data = generate_data(adj_list[graph_ind],graph_ind)
        data_list.append(data)
loader = DataLoader(data_list, batch_size=32, shuffle=True)

test_data_list = []
for graph_ind in [0,1,2,3,4,5]:
    for i in range(0,20):
        ind = 0
        data = generate_data(adj_list[graph_ind],graph_ind)
        test_data_list.append(data)
test_loader = DataLoader(test_data_list, batch_size=32, shuffle=True)

tensor([[4.0053, 4.2727],
        [2.9637, 2.8162]])
tensor([[3.6199, 4.0901],
        [2.5779, 3.1751]])
tensor([[2.7334, 2.5572],
        [2.7494, 2.6529]])
tensor([[2.7246, 3.3868],
        [3.5043, 3.7736]])
tensor([[3.5908, 3.6982],
        [3.6581, 3.4326]])
tensor([[4.2126, 4.1053],
        [3.3896, 2.5219]])
tensor([[3.6197, 3.3716],
        [3.6092, 3.0721]])
tensor([[3.9217, 2.6058],
        [3.4328, 3.3227]])
tensor([[2.8390, 3.7772],
        [3.4773, 4.0349]])
tensor([[2.8440, 3.1773],
        [3.2548, 3.6533]])
tensor([[3.3871, 4.1714],
        [4.0471, 3.0552]])
tensor([[2.6474, 3.6554],
        [2.9764, 3.5921]])
tensor([[4.0241, 2.7088],
        [3.3527, 2.4931]])
tensor([[3.2771, 4.0895],
        [3.2471, 3.7912]])
tensor([[4.2232, 3.1965],
        [2.4889, 4.1655]])
tensor([[3.1380, 2.9279],
        [4.0932, 3.4067]])
tensor([[3.0360, 2.7863],
        [3.2791, 3.2799]])
tensor([[2.6696, 2.8315],
        [2.6272, 2.4847]])
tensor([[4.1499, 3.3488],
        [2.6768, 3.3

In [32]:
class GCN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = GCNConv(-1, 2)
        self.conv2 = GCNConv(2, 6)
        self.linear = torch.nn.Linear(6, 4)  # Linear layer to output 6x1 vector

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        #x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        # Here we aggregate the output from all nodes
        x = x.mean(dim=0, keepdim=True)
        x = self.linear(x)

        return x

In [39]:
class GCN_new(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = GCNConv(-1, 16)
        self.linear = torch.nn.Linear(16, 16)  # Linear layer to output 6x1 vector
        self.linear2 = torch.nn.Linear(16, 16)  # Linear layer to output 6x1 vector
        self.linear3 = torch.nn.Linear(16, 8)  # Linear layer to output 6x1 vector
        self.linear4 = torch.nn.Linear(8, 4)  # Linear layer to output 6x1 vector

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = x.mean(dim=0, keepdim=True)
        x = self.linear(x)
        x = F.relu(x)
        x = self.linear2(x)
        x = F.relu(x)
        x = self.linear3(x)
        x = F.relu(x)
        x = self.linear4(x)
        # x = F.dropout(x, training=self.training)
        # x = self.conv2(x, edge_index)
        # Here we aggregate the output from all nodes
        


        return x

In [46]:
class lin_net(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = torch.nn.Linear(4, 16)  # Linear layer to output 6x1 vector
        self.linear2 = torch.nn.Linear(16, 16)  # Linear layer to output 6x1 vector
        self.linear3 = torch.nn.Linear(16, 8)  # Linear layer to output 6x1 vector
        self.linear4 = torch.nn.Linear(8, 4)  # Linear layer to output 6x1 vector

    def forward(self, data):
        x = data.x.t()
        x = torch.flatten(x)
        x = self.linear(x)
        x = F.relu(x)
        x = self.linear2(x)
        x = F.relu(x)
        x = self.linear3(x)
        x = F.relu(x)
        x = self.linear4(x)
        # x = F.dropout(x, training=self.training)
        # x = self.conv2(x, edge_index)
        # Here we aggregate the output from all nodes
        


        return x

In [47]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = lin_net().to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

model.train()
for epoch in range(50):
    loss = 0
    for data_point in data_list:
        data = data_point.to(device)
        optimizer.zero_grad()
        out = model(data)
        loss_fn = torch.nn.MSELoss()
        target = data.y.float()
        loss = loss + loss_fn(out.float(), target)
        loss = loss.float()

    print(epoch)
    print(loss)
    loss.backward()
    optimizer.step()


C:\Users\JansenKlompLF\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\torch\nn\modules\loss.py:536: UserWarning: Using a target size (torch.Size([4])) that is different to the input size (torch.Size([1, 4])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


0
tensor(299.4535, device='cuda:0', grad_fn=<AddBackward0>)
1
tensor(276.0065, device='cuda:0', grad_fn=<AddBackward0>)
2
tensor(254.5883, device='cuda:0', grad_fn=<AddBackward0>)
3
tensor(233.2863, device='cuda:0', grad_fn=<AddBackward0>)
4
tensor(211.7639, device='cuda:0', grad_fn=<AddBackward0>)
5
tensor(190.5243, device='cuda:0', grad_fn=<AddBackward0>)
6
tensor(170.5146, device='cuda:0', grad_fn=<AddBackward0>)
7
tensor(152.4123, device='cuda:0', grad_fn=<AddBackward0>)
8
tensor(137.2422, device='cuda:0', grad_fn=<AddBackward0>)
9
tensor(126.0255, device='cuda:0', grad_fn=<AddBackward0>)
10
tensor(118.6675, device='cuda:0', grad_fn=<AddBackward0>)
11
tensor(112.6578, device='cuda:0', grad_fn=<AddBackward0>)
12
tensor(105.9478, device='cuda:0', grad_fn=<AddBackward0>)
13
tensor(97.9321, device='cuda:0', grad_fn=<AddBackward0>)
14
tensor(89.5232, device='cuda:0', grad_fn=<AddBackward0>)
15
tensor(81.8146, device='cuda:0', grad_fn=<AddBackward0>)
16
tensor(75.1843, device='cuda:0', g

In [48]:
correct = 0
total = 0
for data_point in test_data_list:
    data = data_point.to(device)
    out = model(data)
    print(out)
    print(data.y)

tensor([[0.0203, 0.6893, 0.3265, 0.0033]], device='cuda:0',
       grad_fn=<AddmmBackward0>)
tensor([0., 0., 0., 0.], device='cuda:0', dtype=torch.float64)
tensor([[0.0117, 0.6741, 0.3097, 0.0053]], device='cuda:0',
       grad_fn=<AddmmBackward0>)
tensor([0., 0., 0., 0.], device='cuda:0', dtype=torch.float64)
tensor([[0.0036, 0.6636, 0.3098, 0.0056]], device='cuda:0',
       grad_fn=<AddmmBackward0>)
tensor([0., 0., 0., 0.], device='cuda:0', dtype=torch.float64)
tensor([[0.0204, 0.7796, 0.5152, 0.0027]], device='cuda:0',
       grad_fn=<AddmmBackward0>)
tensor([0., 0., 0., 0.], device='cuda:0', dtype=torch.float64)
tensor([[0.0107, 0.6722, 0.3076, 0.0055]], device='cuda:0',
       grad_fn=<AddmmBackward0>)
tensor([0., 0., 0., 0.], device='cuda:0', dtype=torch.float64)
tensor([[0.0576, 0.7910, 0.3719, 0.0049]], device='cuda:0',
       grad_fn=<AddmmBackward0>)
tensor([0., 0., 0., 0.], device='cuda:0', dtype=torch.float64)
tensor([[0.0357, 0.7284, 0.3475, 0.0031]], device='cuda:0',
    